In [113]:
from bs4 import BeautifulSoup
import pandas as pd

# Create an empty DataFrame
df = pd.DataFrame()

for grupo_produto in ['vestido', 'top-blusa', 'calca', 'saia', 'short', 'camisa', 'blazer']:

    # Read the file
    with open(f'dataset_images/animale-{grupo_produto}.html', 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Extract and print the src attribute of the first img element in each line
    products = []
    for line in lines:
        product = {}
        soup = BeautifulSoup(line, 'html.parser')
        span = soup.find('span', class_='styles_productShelfFlagNew__uxT_L')
        if span:
            product['shelf'] = span.text
        a = soup.find('a', class_='styles_productShelfImageLink__YRO04')
        if a:
            product['href'] = a['href']
        # get all the multiple styles_productShelfImage__AT9cG img elements
        imgs = soup.find_all('img', class_='styles_productShelfImage__AT9cG')
        product['imgs'] = [img['src'] for img in imgs]

        h3 = soup.find('h3', class_='styles_productShelfName__1wpBX')
        if h3:
            product['descricao'] = h3.text
        span = soup.find('span', class_='styles_productShelfPrice__J5a8R')
        if span:
            product['preco'] = span.text
        
        product["grupo"] = f"{grupo_produto}"

        products.append(product)

    shape_antes = df.shape
    # Concat the list of dictionaries into a DataFrame
    df = pd.concat([df, pd.DataFrame(products)])

    shape_atual = df.shape

    # Corrigindo o erro de IndexError com uma verificação de lista não vazia
    df["link 1° imagem"] = df["imgs"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
    df["link produto"] = "https://www.animale.com.br" + df["href"]
    print(f"quantidade de produtos em {grupo_produto}: {shape_atual[0]-shape_antes[0]}")

quantidade de produtos em vestido: 219
quantidade de produtos em top-blusa: 300
quantidade de produtos em calca: 177
quantidade de produtos em saia: 81
quantidade de produtos em short: 43
quantidade de produtos em camisa: 54
quantidade de produtos em blazer: 19


In [114]:
# Quantidade total de produtos
print(f"quantidade total de produtos: {df.shape}")

quantidade total de produtos: (893, 8)


### Tagueamento de características

In [115]:
# mostrar linha inteira
pd.set_option('display.max_colwidth', None)

In [116]:
df['href'].sample(3)

170        /calca-de-couro-cintura-marcada-preto-preto-25-34-1575-0005-307505/p
122    /calca-de-sarja-black-loose-com-pespontos-preto-25-34-1755-0005-314838/p
257                        /saia-midi-de-organza-preto-52-13-4944-0005-296913/p
Name: href, dtype: object

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 893 entries, 0 to 18
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   shelf           892 non-null    object
 1   href            892 non-null    object
 2   imgs            893 non-null    object
 3   descricao       892 non-null    object
 4   preco           892 non-null    object
 5   grupo           893 non-null    object
 6   link 1° imagem  892 non-null    object
 7   link produto    892 non-null    object
dtypes: object(8)
memory usage: 62.8+ KB


In [118]:
# Tirar href nulos
df = df.dropna(subset=['href'])

In [119]:
# Transformando campo descritivo
df["descricao"] = df["descricao"].apply(lambda x: x.lower())

# Tirar espaços do começo e do final
df["descricao"] = df["descricao"].apply(lambda x: x.strip())

# Substituir espaços por -
df["descricao"] = df["descricao"].apply(lambda x: x.replace(" ", "-"))

# Substituir -- por -
df["descricao"] = df["descricao"].apply(lambda x: x.replace("--", "-"))

# Substituinto caracteres especiais
df["descricao"] = df["descricao"].apply(lambda x: x.replace("ç", "c"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("ã", "a"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("á", "a"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("é", "e"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("í", "i"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("ó", "o"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("õ", "o"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("ú", "u"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("â", "a"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("ê", "e"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("î", "i"))
df["descricao"] = df["descricao"].apply(lambda x: x.replace("ô", "o"))


df.sample(3)

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto
33,VERÃO 25,/saia-midi-off-white-com-abertura-lateral-off-white-25-11-1183-0024-309016/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1794805-527-790/25111183_0024_1-SAIA-MIDI-OFF-WHITE-COM-ABERTURA-LATERAL.jpg?v=638560756948500000, https://lojaanimale.vtexassets.com/arquivos/ids/1794825-527-790/25111183_0024_2-SAIA-MIDI-OFF-WHITE-COM-ABERTURA-LATERAL.jpg?v=638560757059230000, https://lojaanimale.vtexassets.com/arquivos/ids/1794840-527-790/25111183_0024_3-SAIA-MIDI-OFF-WHITE-COM-ABERTURA-LATERAL.jpg?v=638560757144030000, https://lojaanimale.vtexassets.com/arquivos/ids/1794862-527-790/25111183_0024_4-SAIA-MIDI-OFF-WHITE-COM-ABERTURA-LATERAL.jpg?v=638560757260470000]",saia-midi-off-white-com-abertura-lateral,"R$ 898,00",saia,https://lojaanimale.vtexassets.com/arquivos/ids/1794805-527-790/25111183_0024_1-SAIA-MIDI-OFF-WHITE-COM-ABERTURA-LATERAL.jpg?v=638560756948500000,https://www.animale.com.br/saia-midi-off-white-com-abertura-lateral-off-white-25-11-1183-0024-309016/p
19,VERÃO 25,/saia-midi-de-tricot-preta-com-torcao-preto-25-11-1215-0005-311040/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1811278-527-790/25111215_0005_1-SAIA-MIDI-DE-TRICOT-PRETA-COM-TORCAO.jpg?v=638585703275400000, https://lojaanimale.vtexassets.com/arquivos/ids/1811298-527-790/25111215_0005_2-SAIA-MIDI-DE-TRICOT-PRETA-COM-TORCAO.jpg?v=638585703387730000, https://lojaanimale.vtexassets.com/arquivos/ids/1811316-527-790/25111215_0005_3-SAIA-MIDI-DE-TRICOT-PRETA-COM-TORCAO.jpg?v=638585703496200000, https://lojaanimale.vtexassets.com/arquivos/ids/1811329-527-790/25111215_0005_4-SAIA-MIDI-DE-TRICOT-PRETA-COM-TORCAO.jpg?v=638585703567730000]",saia-midi-de-tricot-preta-com-torcao,"R$ 998,00",saia,https://lojaanimale.vtexassets.com/arquivos/ids/1811278-527-790/25111215_0005_1-SAIA-MIDI-DE-TRICOT-PRETA-COM-TORCAO.jpg?v=638585703275400000,https://www.animale.com.br/saia-midi-de-tricot-preta-com-torcao-preto-25-11-1215-0005-311040/p
20,VERÃO 25,/calca-detalhe-amarracao-na-barra-rosa-rosa-shocking-25-34-2006-09283-320974/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1823418-527-790/25342006_09283_1-CALCA-DETALHE-AMARRACAO-NA-BARRA-ROSA.jpg?v=638640923543670000, https://lojaanimale.vtexassets.com/arquivos/ids/1823440-527-790/25342006_09283_2-CALCA-DETALHE-AMARRACAO-NA-BARRA-ROSA.jpg?v=638640923678770000, https://lojaanimale.vtexassets.com/arquivos/ids/1823464-527-790/25342006_09283_3-CALCA-DETALHE-AMARRACAO-NA-BARRA-ROSA.jpg?v=638640923834700000, https://lojaanimale.vtexassets.com/arquivos/ids/1823482-527-790/25342006_09283_4-CALCA-DETALHE-AMARRACAO-NA-BARRA-ROSA.jpg?v=638640923913730000]",calca-detalhe-amarracao-na-barra-rosa,"R$ 998,00",calca,https://lojaanimale.vtexassets.com/arquivos/ids/1823418-527-790/25342006_09283_1-CALCA-DETALHE-AMARRACAO-NA-BARRA-ROSA.jpg?v=638640923543670000,https://www.animale.com.br/calca-detalhe-amarracao-na-barra-rosa-rosa-shocking-25-34-2006-09283-320974/p


In [120]:
# Mostrar linhas que tem / na descrição
df[df["descricao"].str.contains("/")]

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto
57,VERÃO 25,/blusa-de-georgette-manga-7-8-babados-estampado-est-pois-grafico-color-52-13-5371-09561-311595/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1818547-527-790/52135371_09561_1-BLUSA-DE-GEORGETTE-MANGA-7-8-BABADOS-ESTAMPADO.jpg?v=638602810504230000, https://lojaanimale.vtexassets.com/arquivos/ids/1818561-527-790/52135371_09561_2-BLUSA-DE-GEORGETTE-MANGA-7-8-BABADOS-ESTAMPADO.jpg?v=638602810593630000, https://lojaanimale.vtexassets.com/arquivos/ids/1818579-527-790/52135371_09561_3-BLUSA-DE-GEORGETTE-MANGA-7-8-BABADOS-ESTAMPADO.jpg?v=638602810696300000, https://lojaanimale.vtexassets.com/arquivos/ids/1818594-527-790/52135371_09561_4-BLUSA-DE-GEORGETTE-MANGA-7-8-BABADOS-ESTAMPADO.jpg?v=638602810785800000]",blusa-de-georgette-manga-7/8-babados-estampado,"R$ 1.198,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/ids/1818547-527-790/52135371_09561_1-BLUSA-DE-GEORGETTE-MANGA-7-8-BABADOS-ESTAMPADO.jpg?v=638602810504230000,https://www.animale.com.br/blusa-de-georgette-manga-7-8-babados-estampado-est-pois-grafico-color-52-13-5371-09561-311595/p
16,VERÃO 25,/camisa-manga-7-8-marinho-com-martingale-marinho-52-05-1517-1974-314915/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1819972-527-790/52051517_1974_1-CAMISA-MANGA-7-8-MARINHO-COM-MARTINGALE.jpg?v=638627205543170000, https://lojaanimale.vtexassets.com/arquivos/ids/1819986-527-790/52051517_1974_2-CAMISA-MANGA-7-8-MARINHO-COM-MARTINGALE.jpg?v=638627205649270000, https://lojaanimale.vtexassets.com/arquivos/ids/1820001-527-790/52051517_1974_3-CAMISA-MANGA-7-8-MARINHO-COM-MARTINGALE.jpg?v=638627205773570000, https://lojaanimale.vtexassets.com/arquivos/ids/1820016-527-790/52051517_1974_4-CAMISA-MANGA-7-8-MARINHO-COM-MARTINGALE.jpg?v=638627205859030000, https://lojaanimale.vtexassets.com/arquivos/ids/1820024-527-790/52051517_1974_5-CAMISA-MANGA-7-8-MARINHO-COM-MARTINGALE.jpg?v=638627205939000000]",camisa-manga-7/8-marinho-com-martingale,"R$ 698,00",camisa,https://lojaanimale.vtexassets.com/arquivos/ids/1819972-527-790/52051517_1974_1-CAMISA-MANGA-7-8-MARINHO-COM-MARTINGALE.jpg?v=638627205543170000,https://www.animale.com.br/camisa-manga-7-8-marinho-com-martingale-marinho-52-05-1517-1974-314915/p


In [121]:
# Substituir / por -
df["descricao"] = df["descricao"].apply(lambda x: x.replace("/", "-"))

In [122]:
# Mostrar linhas que tem / na descrição
df[df["descricao"].str.contains("'")]

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto
110,VERÃO 25,/calca-de-sarja-boot-70-s-bege-whitecap-25-34-1821-0030-316737/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1790405-527-790/25341821_0030_1-CALCA-DE-SARJA-BOOT-70-S.jpg?v=638556415524700000, https://lojaanimale.vtexassets.com/arquivos/ids/1790421-527-790/25341821_0030_2-CALCA-DE-SARJA-BOOT-70-S.jpg?v=638556415607470000, https://lojaanimale.vtexassets.com/arquivos/ids/1790438-527-790/25341821_0030_3-CALCA-DE-SARJA-BOOT-70-S.jpg?v=638556415695370000, https://lojaanimale.vtexassets.com/arquivos/ids/1790446-527-790/25341821_0030_4-CALCA-DE-SARJA-BOOT-70-S.jpg?v=638556415791500000]",calca-de-sarja-boot-70's,"R$ 698,00",calca,https://lojaanimale.vtexassets.com/arquivos/ids/1790405-527-790/25341821_0030_1-CALCA-DE-SARJA-BOOT-70-S.jpg?v=638556415524700000,https://www.animale.com.br/calca-de-sarja-boot-70-s-bege-whitecap-25-34-1821-0030-316737/p
151,VERÃO 25,/calca-jeans-boot-70-s-com-resina-transparente-jeans-claro-25-34-1773-0101-315615/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1780508-527-790/25341773_0101_1-CALCA-JEANS-BOOT-70S-COM-RESINA-TRANSPAR.jpg?v=638539866376730000, https://lojaanimale.vtexassets.com/arquivos/ids/1780527-527-790/25341773_0101_2-CALCA-JEANS-BOOT-70S-COM-RESINA-TRANSPAR.jpg?v=638539866507730000, https://lojaanimale.vtexassets.com/arquivos/ids/1780551-527-790/25341773_0101_3-CALCA-JEANS-BOOT-70S-COM-RESINA-TRANSPAR.jpg?v=638539866643430000, https://lojaanimale.vtexassets.com/arquivos/ids/1780564-527-790/25341773_0101_4-CALCA-JEANS-BOOT-70S-COM-RESINA-TRANSPAR.jpg?v=638539866734070000]",calca-jeans-boot-70's-com-resina-transparente,"R$ 728,00",calca,https://lojaanimale.vtexassets.com/arquivos/ids/1780508-527-790/25341773_0101_1-CALCA-JEANS-BOOT-70S-COM-RESINA-TRANSPAR.jpg?v=638539866376730000,https://www.animale.com.br/calca-jeans-boot-70-s-com-resina-transparente-jeans-claro-25-34-1773-0101-315615/p


In [123]:
# Substituir ' por -
df["descricao"] = df["descricao"].apply(lambda x: x.replace("'", "-"))

In [124]:
# Mostrar linhas que têm "?" na descrição
df[df["descricao"].str.contains(r"\?", na=False)]


,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto
20,VERÃO 25,/top-de-alca-verde-exotique-verde-exotique-52-10-5285-5339-325786/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1826237-527-790/52105285_5339_1-TOP-DE-ALCA-VERDE-EXOTIQUE--.jpg?v=638645288690100000, https://lojaanimale.vtexassets.com/arquivos/ids/1826254-527-790/52105285_5339_2-TOP-DE-ALCA-VERDE-EXOTIQUE--.jpg?v=638645288789100000, https://lojaanimale.vtexassets.com/arquivos/ids/1826266-527-790/52105285_5339_3-TOP-DE-ALCA-VERDE-EXOTIQUE--.jpg?v=638645288884270000, https://lojaanimale.vtexassets.com/arquivos/ids/1826280-527-790/52105285_5339_4-TOP-DE-ALCA-VERDE-EXOTIQUE--.jpg?v=638645288964730000]",top-de-alca-verde-exotique-?,"R$ 898,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/ids/1826237-527-790/52105285_5339_1-TOP-DE-ALCA-VERDE-EXOTIQUE--.jpg?v=638645288690100000,https://www.animale.com.br/top-de-alca-verde-exotique-verde-exotique-52-10-5285-5339-325786/p
2,LANÇAMENTO,/calca-alfaiataria-linho-marrom-bordo-marrom-bordo-25-34-2037-09281-321772/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1823631-527-790/25342037_09281_1-CALCA-ALFAIATARIA-LINHO-MARROM-BORDO--.jpg?v=638640924743470000, https://lojaanimale.vtexassets.com/arquivos/ids/1823650-527-790/25342037_09281_2-CALCA-ALFAIATARIA-LINHO-MARROM-BORDO--.jpg?v=638640924849430000, https://lojaanimale.vtexassets.com/arquivos/ids/1823670-527-790/25342037_09281_3-CALCA-ALFAIATARIA-LINHO-MARROM-BORDO--.jpg?v=638640924956230000, https://lojaanimale.vtexassets.com/arquivos/ids/1823685-527-790/25342037_09281_4-CALCA-ALFAIATARIA-LINHO-MARROM-BORDO--.jpg?v=638640925051930000]",calca-alfaiataria-linho-marrom-bordo-?,"R$ 1.198,00",calca,https://lojaanimale.vtexassets.com/arquivos/ids/1823631-527-790/25342037_09281_1-CALCA-ALFAIATARIA-LINHO-MARROM-BORDO--.jpg?v=638640924743470000,https://www.animale.com.br/calca-alfaiataria-linho-marrom-bordo-marrom-bordo-25-34-2037-09281-321772/p


In [125]:
# Substituir -? por -
df["descricao"] = df["descricao"].apply(lambda x: x.replace("?", ""))

# Caso descricao termine com um - tirar
df["descricao"] = df["descricao"].apply(lambda x: x[:-1] if x[-1] == "-" else x)

In [126]:
# Extraindo ID do produto

# href termina no formato -07-20-6810-0005-319446/p
# pegar do -5 até o -1
df["id"] = df["href"].apply(lambda x: x.split("-")[-5:-1])
df["id"] = df["id"].apply(lambda x: "-".join(x))

In [127]:
df['id'].sample(3)

18     03-04-0431-09041
10      03-04-0461-0005
125    52-13-5406-09050
Name: id, dtype: object

In [128]:
# Extraindo codigo 3_____
df["codigo"] = df["href"].apply(lambda x: x.split("-")[-1])

In [129]:
df.sample(1)

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto,id,codigo
23,VERÃO 25,/vestido-de-tricot-midi-detalhe-ondas-coloridas-preto-laranja-turquesa-07-20-6744-00005-319929/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1824948-527-790/07206744_00005_1-VESTIDO-DE-TRICOT-MIDI-DETALHE-ONDAS-COLORIDAS.jpg?v=638645275326100000, https://lojaanimale.vtexassets.com/arquivos/ids/1824958-527-790/07206744_00005_2-VESTIDO-DE-TRICOT-MIDI-DETALHE-ONDAS-COLORIDAS.jpg?v=638645275397870000, https://lojaanimale.vtexassets.com/arquivos/ids/1824971-527-790/07206744_00005_3-VESTIDO-DE-TRICOT-MIDI-DETALHE-ONDAS-COLORIDAS.jpg?v=638645275466070000, https://lojaanimale.vtexassets.com/arquivos/ids/1824979-527-790/07206744_00005_4-VESTIDO-DE-TRICOT-MIDI-DETALHE-ONDAS-COLORIDAS.jpg?v=638645275532500000]",vestido-de-tricot-midi-detalhe-ondas-coloridas,"R$ 898,00",vestido,https://lojaanimale.vtexassets.com/arquivos/ids/1824948-527-790/07206744_00005_1-VESTIDO-DE-TRICOT-MIDI-DETALHE-ONDAS-COLORIDAS.jpg?v=638645275326100000,https://www.animale.com.br/vestido-de-tricot-midi-detalhe-ondas-coloridas-preto-laranja-turquesa-07-20-6744-00005-319929/p,07-20-6744-00005,319929/p


In [130]:
import re

# Extraindo cor/estampa
# href tem o formato /{descricao}-{cor}-{id}-{codigo}

# Função para extrair a cor usando regex
def extrair_cor(href, descricao, id, codigo):
    # Substitui descricao, id e codigo no href
    href = href.replace(f'/{descricao}-', '').replace(f'-{id}-{codigo}', '')
    return href

# Aplicando a função para cada linha do DataFrame
df['cor'] = df.apply(lambda row: extrair_cor(row['href'], row['descricao'], row['id'], row['codigo']), axis=1)

In [131]:
# exibindo quem não começa com est em cor
df[~df["cor"].str.startswith("est")]['cor'].unique()

array(['verde-musgo', 'verde', 'azul-medio', 'bege-flan', 'bege-reflexo',
       'bege-toast', 'vermelho-fiery', 'preto', 'rosa-shocking',
       'laranja-papaya', 'marrom-bordo', 'preto-laranja-turquesa',
       'coral-camelia', 'verde-oliva', 'roxo-mirtilo', 'roxo-fada',
       'bege-kirli', 'marrom-biscoito', 'azul-print', 'marinho',
       'azul-hortensia', 'azul-blue', 'off-white---azul-print',
       'azul-oceanico', 'verde-natureza', 'verde-folha', 'verde-simply',
       'marrom-nozes', 'off-white', 'natural', 'rosa-mosqueta',
       'azul-hortencia', 'laranja-vibrante', 'floral-carimbo-golden',
       'verde-encanto---bronze---rosa-mosqueta', 'verde-encanto',
       'vermelho-argila', 'preto---rosa', 'azul-cannes', 'vinho',
       'amarelo-papiro', 'nude-marzipan', 'bege-bergelim',
       'bege-whitecap', 'branco--preto--rosa', 'dourado',
       'off-white---rosa-mosqueta', 'rose-gold', 'laranja-aura',
       'amarelo-golden', 'bege', 'preto---azul-cannes', 'rosa',
       'lila

In [132]:
df[df["cor"].str.startswith("est")]['cor'].unique()

array(['est-abstrato-silhuetas-yucca', 'est-floral-vagalume-color',
       'est-onca-leve-flan', 'est-pois-ciclos-golden-e-musgo',
       'est-loc-abstrato-solsticio-p', 'est--lenco-pois-recote-g',
       'est-onca-solsticio', 'est-listra-encontro-kirli-e-fiery',
       'est-geo-luas-fiery-p', 'est-listra-alquimia-print-e-kirli',
       'est-pois-grafic-color', 'est-pois-grafico-color',
       'est-floral-sketch-color', 'est-pois-passaro-marinho-p',
       'est-floral-leve-color', 'est-floral-cura-color-m', 'est-leopardo',
       'est-cobra-biscione-natureza', 'est-listra-cura-cogumelo-e-aura',
       'est-onca-aquarela-forte-color-p',
       'est-floral-garden-rum-e-vivo-m', 'est-onca-nite-strawberry-p',
       'est-onca-moderna-color', 'est-onca-nite-color-p',
       'est-liberty-camomila-preto-p', 'est-geo-caminho',
       'est-listra-bicolor-narciso-e-sandalo-p',
       'est-floral-pastel-preto-e-leve',
       'est-floral-geometrico-block-color',
       'est-xadrez-daisy-recorte-co

In [133]:
df.sample(3)

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto,id,codigo,cor
93,VERÃO 25,/calca-jeans-skinny-basic-high-delave-power-jeans-claro-25-34-1778-0101-315644/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1811937-527-790/25341778_0101_1-CALCA-JEANS-SKINNY-BASIC-HIGH-DELAVE-POWER.jpg?v=638585707090200000, https://lojaanimale.vtexassets.com/arquivos/ids/1811949-527-790/25341778_0101_2-CALCA-JEANS-SKINNY-BASIC-HIGH-DELAVE-POWER.jpg?v=638585707161830000, https://lojaanimale.vtexassets.com/arquivos/ids/1811970-527-790/25341778_0101_3-CALCA-JEANS-SKINNY-BASIC-HIGH-DELAVE-POWER.jpg?v=638585707289200000, https://lojaanimale.vtexassets.com/arquivos/ids/1811987-527-790/25341778_0101_4-CALCA-JEANS-SKINNY-BASIC-HIGH-DELAVE-POWER.jpg?v=638585707378770000]",calca-jeans-skinny-basic-high-delave-power,"R$ 548,00",calca,https://lojaanimale.vtexassets.com/arquivos/ids/1811937-527-790/25341778_0101_1-CALCA-JEANS-SKINNY-BASIC-HIGH-DELAVE-POWER.jpg?v=638585707090200000,https://www.animale.com.br/calca-jeans-skinny-basic-high-delave-power-jeans-claro-25-34-1778-0101-315644/p,25-34-1778-0101,315644/p,jeans-claro
44,VERÃO 25,/calca-reta-marinho-com-cinto-marinho-25-34-1617-1974-314777/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1819960-527-790/25341617_1974_1-CALCA-RETA-MARINHO-COM-CINTO.jpg?v=638627205361570000, https://lojaanimale.vtexassets.com/arquivos/ids/1819967-527-790/25341617_1974_2-CALCA-RETA-MARINHO-COM-CINTO.jpg?v=638627205510830000, https://lojaanimale.vtexassets.com/arquivos/ids/1819981-527-790/25341617_1974_3-CALCA-RETA-MARINHO-COM-CINTO.jpg?v=638627205617600000, https://lojaanimale.vtexassets.com/arquivos/ids/1819995-527-790/25341617_1974_4-CALCA-RETA-MARINHO-COM-CINTO.jpg?v=638627205724170000]",calca-reta-marinho-com-cinto,"R$ 798,00",calca,https://lojaanimale.vtexassets.com/arquivos/ids/1819960-527-790/25341617_1974_1-CALCA-RETA-MARINHO-COM-CINTO.jpg?v=638627205361570000,https://www.animale.com.br/calca-reta-marinho-com-cinto-marinho-25-34-1617-1974-314777/p,25-34-1617-1974,314777/p,marinho
49,VERÃO 25,/ragata-de-tricot-vermelha-um-ombro-com-peplum-vermelho-fiery-52-15-1331-8259-310696/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1818773-527-790/52151331_8259_1-RAGATA-DE-TRICOT-VERMELHA-UM-OMBRO-COM-PEPLUM.jpg?v=638602811822870000, https://lojaanimale.vtexassets.com/arquivos/ids/1818784-527-790/52151331_8259_2-RAGATA-DE-TRICOT-VERMELHA-UM-OMBRO-COM-PEPLUM.jpg?v=638602811892770000, https://lojaanimale.vtexassets.com/arquivos/ids/1818795-527-790/52151331_8259_3-RAGATA-DE-TRICOT-VERMELHA-UM-OMBRO-COM-PEPLUM.jpg?v=638602812005170000, https://lojaanimale.vtexassets.com/arquivos/ids/1818809-527-790/52151331_8259_4-RAGATA-DE-TRICOT-VERMELHA-UM-OMBRO-COM-PEPLUM.jpg?v=638602812114330000]",ragata-de-tricot-vermelha-um-ombro-com-peplum,"R$ 398,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/ids/1818773-527-790/52151331_8259_1-RAGATA-DE-TRICOT-VERMELHA-UM-OMBRO-COM-PEPLUM.jpg?v=638602811822870000,https://www.animale.com.br/ragata-de-tricot-vermelha-um-ombro-com-peplum-vermelho-fiery-52-15-1331-8259-310696/p,52-15-1331-8259,310696/p,vermelho-fiery


Fazer um regex depois pra pegar estampa.\
Se tiver 'listra' em descricao estampa = listrado

In [134]:
df.sample(5)

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto,id,codigo,cor
110,VERÃO 25,/t-shirt-de-malha-estampada-com-decote-v-est-onca-nite-color-p-52-06-0700-09558-315907/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1795894-527-790/52060700_09558_1-T-SHIRT-DE-MALHA-ESTAMPADA-COM-DECOTE-V.jpg?v=638560762956000000, https://lojaanimale.vtexassets.com/arquivos/ids/1795905-527-790/52060700_09558_2-T-SHIRT-DE-MALHA-ESTAMPADA-COM-DECOTE-V.jpg?v=638560763017470000, https://lojaanimale.vtexassets.com/arquivos/ids/1795918-527-790/52060700_09558_3-T-SHIRT-DE-MALHA-ESTAMPADA-COM-DECOTE-V.jpg?v=638560763077400000, https://lojaanimale.vtexassets.com/arquivos/ids/1795933-527-790/52060700_09558_4-T-SHIRT-DE-MALHA-ESTAMPADA-COM-DECOTE-V.jpg?v=638560763166070000]",t-shirt-de-malha-estampada-com-decote-v,"R$ 198,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/ids/1795894-527-790/52060700_09558_1-T-SHIRT-DE-MALHA-ESTAMPADA-COM-DECOTE-V.jpg?v=638560762956000000,https://www.animale.com.br/t-shirt-de-malha-estampada-com-decote-v-est-onca-nite-color-p-52-06-0700-09558-315907/p,52-06-0700-09558,315907/p,est-onca-nite-color-p
2,LANÇAMENTO,/vestido-de-georgette-pesado-longo-franzido-estampado-est-abstrato-silhuetas-yucca-07-20-6832-09792-320134/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1825088-527-790/07206832_09792_1-VESTIDO-DE-GEORGETTE-PESADO-LONGO-FRANZIDO-ESTAMPADO.jpg?v=638645276276770000, https://lojaanimale.vtexassets.com/arquivos/ids/1825101-527-790/07206832_09792_2-VESTIDO-DE-GEORGETTE-PESADO-LONGO-FRANZIDO-ESTAMPADO.jpg?v=638645276351900000, https://lojaanimale.vtexassets.com/arquivos/ids/1825120-527-790/07206832_09792_3-VESTIDO-DE-GEORGETTE-PESADO-LONGO-FRANZIDO-ESTAMPADO.jpg?v=638645276468270000, https://lojaanimale.vtexassets.com/arquivos/ids/1825142-527-790/07206832_09792_4-VESTIDO-DE-GEORGETTE-PESADO-LONGO-FRANZIDO-ESTAMPADO.jpg?v=638645276616530000]",vestido-de-georgette-pesado-longo-franzido-estampado,"R$ 1.498,00",vestido,https://lojaanimale.vtexassets.com/arquivos/ids/1825088-527-790/07206832_09792_1-VESTIDO-DE-GEORGETTE-PESADO-LONGO-FRANZIDO-ESTAMPADO.jpg?v=638645276276770000,https://www.animale.com.br/vestido-de-georgette-pesado-longo-franzido-estampado-est-abstrato-silhuetas-yucca-07-20-6832-09792-320134/p,07-20-6832-09792,320134/p,est-abstrato-silhuetas-yucca
5,LANÇAMENTO,/vestido-curto-azul-de-alca-azul-medio-07-20-6952-04502-321657/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1830662-527-790/07206952_04502_1-VESTIDO-CURTO-AZUL-DE-ALCA.jpg?v=638659158839900000, https://lojaanimale.vtexassets.com/arquivos/ids/1830678-527-790/07206952_04502_2-VESTIDO-CURTO-AZUL-DE-ALCA.jpg?v=638659158993330000, https://lojaanimale.vtexassets.com/arquivos/ids/1830694-527-790/07206952_04502_3-VESTIDO-CURTO-AZUL-DE-ALCA.jpg?v=638659159163830000, https://lojaanimale.vtexassets.com/arquivos/ids/1830710-527-790/07206952_04502_4-VESTIDO-CURTO-AZUL-DE-ALCA.jpg?v=638659159323930000]",vestido-curto-azul-de-alca,"R$ 1.198,00",vestido,https://lojaanimale.vtexassets.com/arquivos/ids/1830662-527-790/07206952_04502_1-VESTIDO-CURTO-AZUL-DE-ALCA.jpg?v=638659158839900000,https://www.animale.com.br/vestido-curto-azul-de-alca-azul-medio-07-20-6952-04502-321657/p,07-20-6952-04502,321657/p,azul-medio
129,VERÃO 25,/regata-de-tricot-com-decote-nadador-basico-off-white-52-15-1339-0003-308941/p,"[https://lojaanimale.vtexassets.com/arquivos/ids/1796532-527-790/52151339_0003_1-REGATA-DE-TRICOT-COM-DECOTE-NADADOR-BASICO.jpg?v=638560776559730000, https://lojaanimale.vtexassets.com/arquivos/ids/1796563-527-790/52151339_0003_2-REGATA-DE-TRICOT-COM-DECOTE-NADADOR-BASICO.jpg?v=638560776715970000, https://lojaanimale.vtexassets.com/arquivos/ids/1796583-527-790/52151339_0003_3-REGATA-DE-TRICOT-COM-DECOTE-NADADOR-BASICO.jpg?v=638560776823030000, https://lojaanimale.vtexassets.com/arquivos/ids/1796604-527-790/52151339_0003_4-REGATA-DE-TRICOT-COM-DECOTE-NADADOR-BASICO.jpg?v=638560776933030000]",regata-de-tricot-com-decote-nadador-basico,"R$ 4

### Baixando imagens

In [136]:
# Baixando imagens dos produtos (a partir de link 1° imagem) com o titulo da imagem no formatodescricao_cor_id
import requests
import os

# Criar pasta para salvar as imagens
os.makedirs('animale_images', exist_ok=True)

# Função para baixar a imagem
def baixar_imagem(url, pasta, nome):
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'{pasta}/{nome}', 'wb') as f:
            f.write(response.content)

# Baixar as imagens
for index, row in df.iterrows():
    # Extraindo informações
    descricao = row['descricao']
    cor = row['cor']
    id = row['id']
    link = row['link 1° imagem']
    grupo = row['grupo']
    # Baixando a imagem
    baixar_imagem(link, 'animale_images', f'{grupo}_{descricao}_{cor}_{id}.jpg')


In [137]:
# Salvando dataframe em um arquivo parquet
df.to_parquet('dataset_images/base_animale.parquet', index=False)